In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 57.3 MB/s 


In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 5.3 MB/s 


In [ ]:
import nltk
nltk.download()

In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
import re
from soynlp.normalizer import *

# 데이터 로드

In [ ]:
path = '/content/gdrive/MyDrive/CUAI/CUAI/하계 프로젝트/'

In [ ]:
data = pd.read_excel(path + 'final_data.xlsx', engine='openpyxl')

In [ ]:
data = data[['document', 'label']]
data.drop_duplicates(subset=['document'], ignore_index=True)
data

,document,label
0,어릴 때 보고 지금 다시 봐도 재밌어ㅋㅋ,1
1,디자인을 배우는 학생으로 외국 디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...,1
2,폴리스 스토리 시리즈는부터 뉴까지 버릴게 하나도 없음 최고,1
3,와 연기가 진짜 개쩔구나 지루할 거라고 생각했는데 몰입해서 봤다 그래 이런 게 진짜...,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...
303470,내가 게을러서 다른 방법을 아직 생각하진 못했어,0
303471,댄스 동호회는 좀 괜찮은 것 같은데 이참에 가입해볼까,0
303472,당장 내일 밥은 어떡하고 내 병원비는 어쩌지,0
303473,응 이제 다들 독립해가지고 명절 아니면 찾아오질 않아,0


In [ ]:
data1 = data['document']
label = data['label']
x_train, x_test, y_train, y_test = train_test_split(data1, label, test_size = 0.25, random_state=42, shuffle = True, stratify = label)

# 형태소 분석
- 불용어 제거 방법  
  - 형태소 분석 시 품사 태깅을 활용하자.
    - 품사 태깅을 진행하고  
     조사, 접사, 접두사, 접미사를 제거하고 단어사전을 생성. 

In [ ]:
okt = Okt()
#부착되는 품사 태그 기호와 의미 확인
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [ ]:
#train_data 형태소 분석
stopwords = ['ㅋ', 'ㅎ', 'ㅠ', 'ㅡ', 'ㅉ']
stoptags = ['Josa', 'Eomi', 'Suffix']
train_data = []

for sent in tqdm(x_train):
    tokenized_sent = okt.pos(sent)
    stopwords_removed_sent = [word[0] for word in tokenized_sent if (not word[1] in stoptags) and (not word[0] in stopwords)]
    train_data.append(stopwords_removed_sent)

100%|██████████| 227606/227606 [11:08<00:00, 340.44it/s]


In [ ]:
#test data 형태소 분석
test_data = []

for sent in tqdm(x_test):
    tokenized_sent = okt.pos(sent)
    stopwords_removed_sent = [word[0] for word in tokenized_sent if (not word[1] in stoptags) and (not word[0] in stopwords)]
    test_data.append(stopwords_removed_sent)

100%|██████████| 75869/75869 [04:16<00:00, 295.91it/s]


# 정수인코딩 & 패딩
- 패딩은 텐서플로 tokenizer의 paddding을 사용 => option은 pre로 설정
- 패딩의 길이
  1. 95%를 포함하는 길이 => 약 28~30
  2. 평균 길이로 해보기

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data) 

In [ ]:
len(tokenizer.word_index) #단어 수

92617

In [ ]:
tokenizer.word_index

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 92617
등장 빈도가 2번 이하인 희귀 단어의 수: 56935
단어 집합에서 희귀 단어의 비율: 61.47359556021033
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.1832513037660277


In [ ]:
vocab_size = total_cnt - rare_cnt + 1
vocab_size

35683

In [ ]:
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(train_data) 

In [ ]:
X_train = tokenizer.texts_to_sequences(train_data)
X_train

In [ ]:
X_test = tokenizer.texts_to_sequences(test_data)

In [ ]:
X_test

In [ ]:
#X_train결측치 확인
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1] 
len(drop_train)

373

In [ ]:
#모두 넘파이 배열로 바꿔준다.
X_train = np.array(X_train)
Y_train = np.array(y_train)
X_test = np.array(X_test)
Y_test = np.array(y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [ ]:
#Train데이터의 결측치 제거
X_train = np.delete(X_train, drop_train, axis=0)
Y_train = np.delete(Y_train, drop_train, axis=0)

In [ ]:
#X_test결측치 확인
drop_test = [index for index, sentence in enumerate(X_test) if len(sentence) < 1] 

#test데이터의 결측치 제거
X_test = np.delete(X_test, drop_test, axis=0)
Y_test = np.delete(Y_test, drop_test, axis=0)

# 에브리타임 강의 데이터

In [ ]:
review_list = pd.read_csv(path + 'review_list.txt', sep = '\t', encoding= 'utf-8', header = None)

In [ ]:
review_list.columns = ['document']

In [ ]:
review_list

,document
0,강의는 이해하기 쉽게 설명하시기 때문에 어려움 없이 공부할 수 있어요행동 수정하는 ...
1,교수님이 유쾌하시고 이야기도 많이 들려주십니다. 과제도 어려운 편 아니고 조금만 집...
2,교수님 너무 좋고 수업도 좋음시험을 잘 본 친구들이 많아서 평균보다 높아도 비쁠나옴...
3,처음엔 좀 헷갈릴 순 있는 내용이지만 교수님께서 차근차근 쉽게 잘 알려주신다. 시험...
4,수업은 너무 잘 해주셨는데공부 열심히 했는데도 점수를 보니 내가 걸러져있었다ㅠ
...,...
6027,뭐.. 제가 드릴말씀은 64명강의에 9명만 A+이라는 점이네요. 교수님이 처음 강의...
6028,무난 무난 무난.토론 과제 준비 과정은 힘들다면 힘들지만 나름 새로웠고 무엇보다 세...
6029,내용은 평이한 수준인데 문제는 수업 진행이 너무 미숙했다. 그냥 학생들이 앞으로 나...
6030,수업에 깊이가 별로 없어요. 그냥 작품해설 한 편 읽어보면 다 알 수 있을 것 같은...


In [ ]:
#중복 제거
review_list1 = review_list.drop_duplicates(subset=['document'], ignore_index=True)

In [ ]:
#맞춤법 검사
!pip install git+https://github.com/ssut/py-hanspell.git 

#맞춤법 검사하는 함수
from hanspell import spell_checker
def spell_check(sent):
  return spell_checker.check(sent).checked

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-cv0h95ay
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-cv0h95ay


In [ ]:
review_list1.loc[0]['document']

'강의는 이해하기 쉽게 설명하시기 때문에 어려움 없이 공부할 수 있어요행동 수정하는 과제가 하나 있는데 쉽게 할 수 있어서 과제 부담은 적습니다시험이 어려운 것은 아니지만 변별력은 있기 때문에 성적 잘 받으려면 공부좀 해야합니다단순 암기보다는 내용 이해가 좋은 점수 얻는데 도움돼요'

In [ ]:
#불용어 제거하는 것
def okt_tag_tokenizer(sent):
  okt = Okt()
  stopwords = ['ㅋ', 'ㅎ', 'ㅠ', 'ㅡ', 'ㅉ']
  stoptags = ['Josa', 'Eomi', 'Suffix']
  tokenized_sent = okt.pos(sent)
  stopwords_removed_sent = [word[0] for word in tokenized_sent if (not word[1] in stoptags) and (not word[0] in stopwords)]
  
  return stopwords_removed_sent

In [ ]:
from nltk.tokenize import sent_tokenize
def review_preprocessing(sent):
  sent = spell_check(sent) #맞춤법 검사
  sent_lst = (sent_tokenize(sent)) #문장 토큰화
  new_sent_lst = []

  for new_sent in sent_lst: #토큰화 & 불용어 제거
    new_sent_lst.append(okt_tag_tokenizer(new_sent))
  
  encoded = tokenizer.texts_to_sequences(new_sent_lst) # 정수 인코딩
  padding = pad_sequences(encoded, maxlen = 30) # 패딩

  return padding

In [ ]:
a = review_preprocessing(review_list1.loc[3]['document'])

In [ ]:
review_list1.loc[3]['document']

'처음엔 좀 헷갈릴 순 있는 내용이지만 교수님께서 차근차근 쉽게 잘 알려주신다. 시험은 어렵진 않지만 변별력이 엄청남..! 그리고 시험 성적 분포가 되게 촘촘하고 동점자도 많은 것 같으니 좋은 성적을 받으려면 꼼꼼하게 공부해야 함.'

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model(path + 'LSTM_2.h5')

In [ ]:
a = np.array(a)
b = a[1].reshape(1, -1)
b.shape

(1, 30)

In [ ]:
a[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,  458, 1450], dtype=int32)

In [ ]:
b

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  458, 1450]], dtype=int32)

In [ ]:
model.predict(b)

array([[0.16027135]], dtype=float32)